# Homework 3: Suggested Solution

# Question 1

# Question 2

In [1]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_titanic_data():
    tarball_path = Path("datasets/titanic.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/titanic.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as titanic_tarball:
            titanic_tarball.extractall(path="datasets")
    return [pd.read_csv(Path("datasets/titanic") / filename) for filename in ("train.csv", "test.csv")]

In [89]:
train_data, test_data = load_titanic_data()

In [90]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen 'Carrie'",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [91]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


The goal is to train a classifier that can predict the *Survived column* based on the other columns. 

However, the test data does *not* contain the labels. 

Your goal is to train the best model you can on the training data, then make your predictions on the test data.

Normally, you can upload your file to Kaggle to see your final score, but you have to consider how you validate your answer when there is no label in the test set.

The attributes have the following meaning:
* **PassengerId**: a unique identifier for each passenger
* **Survived**: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* **Pclass**: passenger class.
* **Name**, **Sex**, **Age**: self-explanatory
* **SibSp**: how many siblings & spouses of the passenger aboard the Titanic.
* **Parch**: how many children & parents of the passenger aboard the Titanic.
* **Ticket**: ticket id
* **Fare**: price paid (in pounds)
* **Cabin**: passenger's cabin number
* **Embarked**: where the passenger embarked the Titanic

The goal is to predict whether or not a passenger survived based on attributes such as their age, sex, passenger class, where they embarked and so on.

Let's explicitly set the `PassengerId` column as the index column: (WHY?)

In [92]:
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")

In [93]:
# Let's get more info to see how much data is missing:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [94]:
train_data[train_data["Sex"]=="female"]["Age"].median()

27.0

Okay, the **Age**, **Cabin** and **Embarked** attributes are sometimes null (less than 891 non-null), especially the **Cabin** (77% are null). We will ignore the **Cabin** for now and focus on the rest. The **Age** attribute has about 19% null values, so we will need to decide what to do with them. Replacing null values with the median age seems reasonable. We could be a bit smarter by predicting the age based on the other columns (for example, the median age is 37 in 1st class, 29 in 2nd class and 24 in 3rd class), but we'll keep things simple and just use the overall median age.

The **Name** and **Ticket** attributes may have some value, but they will be a bit tricky to convert into useful numbers that a model can consume. So for now, we will ignore them.

In [95]:
train_data[['Sex','Ticket']]

,Sex,Ticket
PassengerId,,
1,male,A/5 21171
2,female,PC 17599
3,female,STON/O2. 3101282
4,female,113803
5,male,373450
...,...,...
887,male,211536
888,female,112053
889,female,W./C. 6607


In [96]:
train_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699113,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526507,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.416700,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Yikes, only 38% **Survived**! 😭 That's close enough to 40%, so accuracy will be a reasonable metric to evaluate our model.
* The mean **Fare** was £32.20, which does not seem so expensive (but it was probably a lot of money back then).
* The mean **Age** was less than 30 years old.

In [97]:
# Let's check that the target is indeed 0 or 1
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [98]:
# Now let's take a quick look at all the categorical attributes:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [99]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [100]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

The Embarked attribute tells us where the passenger embarked: C=Cherbourg, Q=Queenstown, S=Southampton.

In [101]:
from sklearn.linear_model import SGDClassifier

Let's apply SGDClassifer.

Note that SGDClassifier only accepts numerical values. So, we need to process two things at least

1. impute missing values
2. select features
    - transform string columns into numbers

This process is called *feature engineering*.

Let's handle the missing values first. Let's replace the missing values with the most frequent values of the corresponding columns.

In [102]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


The problems are: Age, Cabin, and Embarked. You can set the rules on your own but need to supply your intuition. Here is just an example.

1. Age: median
2. Cabin, Embarked: most freq
    - For Cabin, just use the first letter

In [103]:
import numpy as np
from sklearn.impute import SimpleImputer

In [104]:
imp_freq = SimpleImputer(missing_values=np.nan, strategy='median')
train_data[['Age']] = imp_freq.fit_transform(train_data[['Age']])

In [108]:
train_data.Cabin = train_data.Cabin.str[0]

In [113]:
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data[['Cabin','Embarked']] = imp_freq.fit_transform(train_data[['Cabin','Embarked']])

In [114]:
train_data[['Cabin','Embarked']] 

,Cabin,Embarked
PassengerId,,
1,C,S
2,C,C
3,C,S
4,C,S
5,C,S
...,...,...
887,C,S
888,B,S
889,C,S


In [117]:
y_train = train_data.Survived

In [120]:
X_train_num = train_data[['Age','SibSp','Parch','Fare']]
X_train_cat = train_data[['Pclass','Cabin','Embarked']]

In [121]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

In [122]:
X_train_cat = enc.fit_transform(X_train_cat)
X_train_cat = pd.DataFrame(X_train_cat,index = train_data.index)
X_train_cat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
PassengerId,,,,,,,,,,,,,,
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [125]:
X_train = pd.concat([X_train_num,X_train_cat],axis = 1)

In [127]:
X_train.head()

,Age,SibSp,Parch,Fare,0,1,2,3,4,5,6,7,8,9,10,11,12,13
PassengerId,,,,,,,,,,,,,,,,,,
1,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,38.0,1,0,71.2833,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,26.0,0,0,7.9250,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [126]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     891 non-null    float64
 1   SibSp   891 non-null    int64  
 2   Parch   891 non-null    int64  
 3   Fare    891 non-null    float64
 4   0       891 non-null    float64
 5   1       891 non-null    float64
 6   2       891 non-null    float64
 7   3       891 non-null    float64
 8   4       891 non-null    float64
 9   5       891 non-null    float64
 10  6       891 non-null    float64
 11  7       891 non-null    float64
 12  8       891 non-null    float64
 13  9       891 non-null    float64
 14  10      891 non-null    float64
 15  11      891 non-null    float64
 16  12      891 non-null    float64
 17  13      891 non-null    float64
dtypes: float64(16), int64(2)
memory usage: 132.3 KB


In [128]:
sgd_clf = SGDClassifier(random_state=123)
sgd_clf.fit(X_train, y_train)

/Users/yeabinmoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


SGDClassifier(random_state=123)

In [129]:
sgd_clf.score(X_train,y_train)

/Users/yeabinmoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.6711560044893379

In [130]:
from sklearn.model_selection import cross_val_score

In [131]:
score = cross_val_score(sgd_clf, X_train, y_train, cv=3)
score.mean()

/Users/yeabinmoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yeabinmoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yeabinmoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yeabinmoon/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error 

0.6509539842873177

In [61]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     891 non-null    object 
 1   SibSp   891 non-null    object 
 2   Parch   891 non-null    object 
 3   Fare    891 non-null    object 
 4   0       891 non-null    float64
 5   1       891 non-null    float64
 6   2       891 non-null    float64
 7   3       891 non-null    float64
 8   4       891 non-null    float64
 9   5       891 non-null    float64
 10  6       891 non-null    float64
 11  7       891 non-null    float64
 12  8       891 non-null    float64
 13  9       891 non-null    float64
 14  10      891 non-null    float64
 15  11      891 non-null    float64
 16  12      891 non-null    float64
 17  13      891 non-null    float64
 18  14      891 non-null    float64
 19  15      891 non-null    float64
dtypes: float64(16), object(4)
memory usage: 146.2+ KB
